# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [147]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=10,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return np.sum((l_s * (l_c/l_s - (l_c / l_s)**2) + r_s
                       * (r_c / r_s - (r_c / r_s)**2)) / (l_s + r_s), axis=1)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return - np.sum((l_c * np.log(l_c / l_s) + r_c
                        * np.log(r_c / r_s)) / (l_s + r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c / (l_s + r_s), axis=1)
                    + np.max(r_c / (l_s + r_s), axis=1))

    def __get_feature_ids_sqrt(self, n_feature):
        # тут опечатка? над range нельзя сделать shuffle
        # feature_ids = range(n_feature)
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature).astype(int)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature).astype(int)]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # сортировка X, y в разрезе определенной фичи.
        # class_number - кол-во классов
        # cut_size - величина среза.
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        # в зависимости от cut_size выполняется разбиение.
        # r_border_ids - это "граничные sampl-ы" в y.
        # индексы ,где меняется значение.
        # если таковые не найдутся, то прерываемся
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        # в r_border_ids индексы размеположены в отсортированном порядке.
        # r_border_ids[n] - r_border_ids[n-1]
        # eq_el_count - массив вида [1, 0, 0, 0, 1, 0, 0]
        # кодируем классы граничных сэмплов на основе one hot encoding
        # в class_increments получаем сколько какой класс повторялся
        # на границе
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        # по найденной границе строятся разбиения на l,r
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # вычисялем различные Gain по разбиениям и находим наилучшее
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # возвращает наилучшую impurity, и порог разбиения
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if node_id == 0:
            self.tree.clear()

        n_sample, n_feature = x.shape
        y_classes = np.bincount(y)
        y_pred = y_classes.argmax()

        if (depth > self.max_depth) or (n_sample < self.min_samples_split) \
                or (np.unique(y).size == 1) \
                or (y_pred > n_sample * self.sufficient_share):
            self.tree[node_id] = (self.LEAF_TYPE,
                                  y_pred, y_classes / y.shape[0])
            return

        feature_ids = self.get_feature_ids(n_feature)

        thresholds = np.apply_along_axis(self.__find_threshold, 0,
                                         x[:, feature_ids], y).T
        best_feature = np.argmin(thresholds[:, 0])
        gs, threshold = thresholds[best_feature]

        x_l, x_r, y_l, y_r = self.__div_samples(x, y,
                                                feature_ids[best_feature],
                                                threshold)
        if (y_l.size == 0) or (y_r.size == 0):
            self.tree[node_id] = (self.LEAF_TYPE, y_pred, y_classes / n_sample)
            return
        self.tree[node_id] = (self.NON_LEAF_TYPE, feature_ids[best_feature],
                              threshold)
        self.feature_importances_[feature_ids[best_feature]] += gs
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [148]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [149]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [152]:
%time clf.fit(X_train, y_train)

CPU times: user 2.93 ms, sys: 129 µs, total: 3.06 ms
Wall time: 2.4 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [151]:
%time my_clf.fit(X_train, y_train)

CPU times: user 31.6 ms, sys: 152 µs, total: 31.7 ms
Wall time: 31.4 ms


## Проверка качества работы на wine

In [153]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8688034188034188

In [154]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

1.0

## Подготовка данных Speed Dating Data 

In [155]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv', encoding='cp1251')

In [156]:
df = df.iloc[:, :97]

df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
              'order', 'partner', 'age_o', 'race_o', 'pf_o_att',
              'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o',
              'shar_o', 'like_o', 'prob_o', 'met_o', 'field', 'undergra',
              'from', 'zipcode', 'career', 'sports', 'tvsports', 'exercise',
              'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing',
              'reading', 'tv', 'theater', 'movies', 'concerts',
              'music', 'shopping', 'yoga', 'expnum', 'wave'], axis=1)

df = df.dropna(subset=['age', 'imprelig', 'imprace', 'date'])

df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)

df.loc[:, 'mn_sat'] = \
            df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = \
            df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = \
            df.loc[:, 'income'].str.replace(',', '').astype(np.float)

df.loc[:, 'mn_sat'] = df.mn_sat.fillna(df.mn_sat[df.mn_sat.notna()].mean())
df.loc[:, 'tuition'] = df.tuition.fillna(df.tuition[df.tuition.notna()].mean())
df.loc[:, 'income'] = df.income.fillna(df.income[df.income.notna()].mean())

df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
               'amb1_1', 'shar1_1']].sum(axis=1)

df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
           'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                     'fun1_1', 'amb1_1', 'shar1_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100

df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
               'amb2_1', 'shar2_1']].sum(axis=1)

df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
           'amb2_1', 'shar2_1']] = \
    (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
            'amb2_1', 'shar2_1']].T /
     df.loc[:, 'temp_totalsum'].T).T * 100

df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]

    if i != 4:
        feat.remove('shar{}_1'.format(i))

    df = df.drop(feat, axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr',
                                          'samerace'], axis=1)\
                                   .dropna()

df_female.columns = df_female.columns + '_f'

df_female = df_female.drop(['pid_f'], axis=1)

df_pair = df_male.join(df_female.set_index('iid_f'),
                       on='pid',
                       how='inner')

df_pair = df_pair.drop(['iid', 'pid'], axis=1)

X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values


In [157]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [158]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [159]:
%time clf.fit(X_train, y_train)

CPU times: user 118 ms, sys: 7.72 ms, total: 125 ms
Wall time: 123 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [160]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.15 s, sys: 4.03 ms, total: 1.16 s
Wall time: 1.15 s


## Проверка качества работы на Speed Dating Data

In [161]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5897435897435896

In [162]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5227773016753205

## Задание 3

In [163]:
clf_features = np.argsort(clf.feature_importances_)[::-1][:10]
my_clf_features = np.argsort(my_clf.feature_importances_)[::-1][:10]
feature_names = df_pair.columns
clf_features_col = clf.feature_importances_[clf_features]
myclf_features_col = my_clf.feature_importances_[my_clf_features]
feature_rating = {"clf_feature_name": feature_names[clf_features],
                  "clf_feature_importance": clf_features_col,
                  "myclf_feature_name": feature_names[my_clf_features],
                  "myclf_feature_importance": myclf_features_col}
pd.DataFrame(data=feature_rating)

,clf_feature_name,clf_feature_importance,myclf_feature_name,myclf_feature_importance
0,match,0.081130,match,1.157408
1,fun1_1_f,0.032153,goal,0.717537
2,imprelig_f,0.030117,fun2_1,0.694521
3,intel1_1_f,0.028996,fun2_1_f,0.685427
4,fun1_1,0.027505,intel1_1_f,0.641914
5,field_cd_f,0.026954,race,0.621854
6,sinc2_1_f,0.026563,fun1_1_f,0.615590
7,career_c_f,0.023935,sinc3_1,0.561803
8,goal_f,0.023870,age,0.490460
9,amb1_1,0.023785,date,0.466377


## Задание 4

In [103]:
#С помощью GridSearchCV или RandomSearchCV подберите наиболее 
#оптимальные параметры для случайного леса (Выберете 2-3 параметра). 
from sklearn.model_selection import GridSearchCV
from random import randint
from sklearn.model_selection import StratifiedKFold

param_grid = {
    'criterion': ['gini', 'entropy'],
    'min_samples_split': range(2,15),
    'max_features': range(2,15)
}

# Некоторые параметры мы задали не простым перечислением значений, а 
# с помощью распределений.

# Будем делать 200 запусков поиска
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

estimator = DecisionTreeClassifier(random_state=123)
grid_search = GridSearchCV(estimator, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# А дальше, просто .fit()
grid_search.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
       error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_split': range(2, 15), 'max_features': range(2, 15)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [104]:
grid_search.best_params_

{'criterion': 'gini', 'max_features': 5, 'min_samples_split': 14}

In [105]:
grid_search.best_score_

0.607483252773894